<a href="https://colab.research.google.com/github/Asmaaelkashef/FaceMask-Detection-Model/blob/main/Face_Mask_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data


#download

In [ ]:
!pip install kagglehub
!pip install kaggle
# IF big dataset :
#!wget  لينك اي شئ ممكن اعمله داون لود

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("omkargurav/face-mask-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/face-mask-dataset


#load data

In [ ]:
from typing import Counter
import numpy as np
import cv2
import os

x = []
y = []

label = 0

classes = os.listdir('/kaggle/input/face-mask-dataset/data')
for c in classes:
    images_paths = os.listdir(f'/kaggle/input/face-mask-dataset/data/{c}' )
    x.extend(images_paths)
    y.extend([label]*len(images_paths))
    label += 1

x = np.array(x)
y = np.array(y)


In [ ]:
from  tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255 ,
    validation_split=0.2 ,
    rotation_range=40 ,
    width_shift_range=0.2 ,
    height_shift_range=0.2 ,
    shear_range=0.2 ,
    zoom_range=0.2 ,
    horizontal_flip=True ,
    fill_mode='nearest')

In [ ]:
train = datagen.flow_from_directory(
    '/kaggle/input/face-mask-dataset/data',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary', # change to catigorical in multiclass
    subset='training', # no need if the folders is splited into train and test
    shuffle=True
)
val = datagen.flow_from_directory(
    '/kaggle/input/face-mask-dataset/data',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.


In [ ]:
# to read from dataframe directly





#Model

Arch

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
model = Sequential()

# Block 1
model.add(Conv2D(32, (3, 3), padding='same', activation='relu',
                 kernel_regularizer=l2(0.001), input_shape=(128, 128, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Block 2
model.add(Conv2D(64, (3, 3), padding='same', activation='relu',
                 kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Block 3
model.add(Conv2D(128, (3, 3), padding='same', activation='relu',
                 kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))  # لمنع الـ overfitting

# Block 4
model.add(Conv2D(256, (3, 3), padding='same', activation='relu',
                 kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# Fully Connected Layers
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(1, activation='sigmoid'))

# Compile
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 16384)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     4,194,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,586,177 (17.49 MB)

 Trainable params: 4,584,705 (17.49 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
earlystop = EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss')
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1) #يقلل learning rate تلقائيًا لو val_loss مش بيتحسن
checkpoint = ModelCheckpoint('face_mask_cnn.h5', monitor='val_loss', save_best_only=True, verbose=1) #يحفظ أفضل نسخة من النموذج

#patience=2: لو لم تتحسن دقة التحقق لمدة 2 ايبوك يلغي
#لما يوقف التدريب، بيرجع أفضل أوزان وصل لها النموذج قبل ما يبدأ يسوء

In [ ]:
callbacks = [earlystop, lr_scheduler, checkpoint]

history = model.fit(
    train,
    validation_data=val,
    epochs=20,
    callbacks=callbacks
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
 73/189 ━━━━━━━━━━━━━━━━━━━━ 42s 364ms/step - accuracy: 0.6931 - loss: 1.5307

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 304ms/step - accuracy: 0.7327 - loss: 1.4501
Epoch 1: val_loss improved from inf to 2.22212, saving model to face_mask_cnn.h5


189/189 ━━━━━━━━━━━━━━━━━━━━ 85s 379ms/step - accuracy: 0.7329 - loss: 1.4497 - val_accuracy: 0.5066 - val_loss: 2.2221 - learning_rate: 1.0000e-04
Epoch 2/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.8054 - loss: 1.2708
Epoch 2: val_loss improved from 2.22212 to 2.21066, saving model to face_mask_cnn.h5


...

189/189 ━━━━━━━━━━━━━━━━━━━━ 43s 228ms/step - accuracy: 0.9271 - loss: 0.8600 - val_accuracy: 0.9265 - val_loss: 0.8722 - learning_rate: 1.0000e-04
Epoch 13/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.9208 - loss: 0.8425
Epoch 13: val_loss did not improve from 0.87221
189/189 ━━━━━━━━━━━━━━━━━━━━ 44s 233ms/step - accuracy: 0.9208 - loss: 0.8425 - val_accuracy: 0.8623 - val_loss: 1.1088 - learning_rate: 1.0000e-04
Epoch 14/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.9174 - loss: 0.8311
Epoch 14: val_loss improved from 0.87221 to 0.85522, saving model to face_mask_cnn.h5


189/189 ━━━━━━━━━━━━━━━━━━━━ 43s 229ms/step - accuracy: 0.9174 - loss: 0.8310 - val_accuracy: 0.9258 - val_loss: 0.8552 - learning_rate: 1.0000e-04
Epoch 15/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 0.9327 - loss: 0.8016
Epoch 15: val_loss did not improve from 0.85522
189/189 ━━━━━━━━━━━━━━━━━━━━ 44s 235ms/step - accuracy: 0.9327 - loss: 0.8016 - val_accuracy: 0.8742 - val_loss: 0.9658 - learning_rate: 1.0000e-04
Epoch 16/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 0.9291 - loss: 0.7757
Epoch 16: val_loss improved from 0.85522 to 0.73717, saving model to face_mask_cnn.h5


189/189 ━━━━━━━━━━━━━━━━━━━━ 44s 231ms/step - accuracy: 0.9291 - loss: 0.7757 - val_accuracy: 0.9404 - val_loss: 0.7372 - learning_rate: 1.0000e-04
Epoch 17/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - accuracy: 0.9323 - loss: 0.7641
Epoch 17: val_loss did not improve from 0.73717
189/189 ━━━━━━━━━━━━━━━━━━━━ 81s 229ms/step - accuracy: 0.9323 - loss: 0.7641 - val_accuracy: 0.9159 - val_loss: 0.7900 - learning_rate: 1.0000e-04
Epoch 18/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 0.9364 - loss: 0.7379
Epoch 18: val_loss improved from 0.73717 to 0.72409, saving model to face_mask_cnn.h5


189/189 ━━━━━━━━━━━━━━━━━━━━ 44s 235ms/step - accuracy: 0.9364 - loss: 0.7379 - val_accuracy: 0.9384 - val_loss: 0.7241 - learning_rate: 1.0000e-04
Epoch 19/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.9315 - loss: 0.7411
Epoch 19: val_loss did not improve from 0.72409
189/189 ━━━━━━━━━━━━━━━━━━━━ 45s 239ms/step - accuracy: 0.9315 - loss: 0.7411 - val_accuracy: 0.9113 - val_loss: 0.9253 - learning_rate: 1.0000e-04
Epoch 20/20
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 0.9353 - loss: 0.7125
Epoch 20: val_loss improved from 0.72409 to 0.70822, saving model to face_mask_cnn.h5


189/189 ━━━━━━━━━━━━━━━━━━━━ 47s 248ms/step - accuracy: 0.9353 - loss: 0.7125 - val_accuracy: 0.9351 - val_loss: 0.7082 - learning_rate: 1.0000e-04


In [ ]:
model.save('face_mask_model.h5')


In [ ]:
from google.colab import files
files.download('face_mask_model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>